In [40]:
import pickle
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset, TensorDataset
import math
import logging
from datetime import datetime
import torch
import gzip
from scipy.stats import pearsonr
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
import torch
import torch.nn as nn
import torch.optim as optim
criterion = torch.nn.MSELoss()

In [23]:
with open('/kaggle/input/nlpnlp1/bert_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [24]:
def df_clean_test(df):
    df_l = df.values.tolist()
    df_new = []
    c=0
    for i in df_l:
        # print(i)
        c=0
        for j in range(0,len(i)):

            if('main-news' in str(i[j]) or 'main-forum' in str(i[j])):
                # print(i)
                c=1
                continue
        if(c==0):
            if (pd.isnull(i[0]) or pd.isnull(i[1]) or pd.isnull(i[2])):
                continue
            df_new.append(i)
    df_new = pd.DataFrame(df_new, columns = ['id','score','sentence1', 'sentence2'])
    # df_new.to_csv('df_new.csv',index=False)
    return df_new

In [25]:
df_score = pd.read_csv('/kaggle/input/nlpnlp1/test_score.csv', delimiter='\t')

# df = pd.read_csv(pd.compat.StringIO(data), delimiter='\t')df_score
test_score = df_score[['score']].values.tolist()
test_score

[[5.0], [4.75], [5.0], [2.4], [2.75], [2.615]]

In [30]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')

def preprocess_data(sentences, scores):
  encoded_inputs = []
  for sent1, sent2 in sentences:
    s = sent1 + sent2
    encoded_dict = tokenizer(s, max_length=60, padding='max_length', truncation=True, return_tensors='pt')
    encoded_inputs.append(encoded_dict)
  return encoded_inputs, torch.tensor(scores, dtype=torch.float32)


def tokenize_data(sentences, scores):
    input_ids = []
    attention_masks = []
    labels = []

    for sentence_pair, score in zip(sentences, scores):
        input_ids.append(sentence_pair['input_ids'].flatten())
        attention_masks.append(sentence_pair['attention_mask'].flatten())
        labels.append(torch.tensor(score, dtype=torch.float))

    input_ids = torch.stack(input_ids).to(device)
    attention_masks = torch.stack(attention_masks).to(device)
    labels = torch.stack(labels).to(device)

    return TensorDataset(input_ids, attention_masks, labels)

In [47]:
df_test = df_clean_test(pd.read_csv('/kaggle/input/nlpnlp1/sample_demo.csv', delimiter='\t'))      #validation dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_sentence = df_test[['sentence1', 'sentence2']].values.tolist()
batch_size = 8
test_sentence, test_score = preprocess_data(test_sentence, test_score)
test_dataset = tokenize_data(test_sentence, test_score)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


model = loaded_model
running_val_loss = 0.0
predictions = []
targets = []
val_losses = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, targets_batch = batch
        input_ids, attention_mask, targets_batch = input_ids.to(device), attention_mask.to(device), targets_batch.to(device)
#             targets_batch = targets_batch.view(-1, 1)  
        targets.extend(targets_batch.cpu().numpy())

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions.extend(outputs.logits.squeeze(-1).cpu().numpy())
#             logits = np.array([(pred[0]+pred[1])/2 for pred in outputs.logits.squeeze(-1)]).reshape(-1)
        logits = outputs.logits
        logits = logits.mean(dim=1)
        loss = criterion(logits, targets_batch)
        running_val_loss += loss.item() * input_ids.size(0)

epoch_val_loss = running_val_loss / len(test_dataset)
val_losses.append(epoch_val_loss)

#     print(predictions)
#     print(targets)
predictions = np.array([(pred[0]+pred[1])/2 for pred in predictions]).reshape(-1)
targets = np.array(targets).reshape(-1)

pearson_corr = np.corrcoef(predictions, targets)[0, 1]
print(f"Test Loss: {epoch_val_loss:.4f}, Pearson Correlation: {pearson_corr:.4f}")

Test Loss: 2.4660, Pearson Correlation: 0.9929


/tmp/ipykernel_34/46112322.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return encoded_inputs, torch.tensor(scores, dtype=torch.float32)
/tmp/ipykernel_34/46112322.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels.append(torch.tensor(score, dtype=torch.float))
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([6, 1])) that is different to the input size (torch.Size([6])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
